<a href="https://colab.research.google.com/github/psy794/stock_pjtt/blob/master/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
# dict_data = separated_datasets_sectors.pkl 불러오기 (업종별:종목코드 딕셔너리)
dict_data = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/separated_datasets_sectors.pkl', 'rb'))
# sectors_features = sectors_features.pkl 불러오기 (업종별:feature 딕셔너리)
sectors_features = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/sectors_features.pkl', 'rb'))

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import pandas as pd

industries = list(dict_data.keys())  # 전체 업종 선택
window_size = 5  # 윈도우 사이즈를 5로 설정

result_df = pd.DataFrame()  # 결과를 저장할 데이터프레임 초기화

# 각 업종에 대해 모델을 학습하고 테스트
for industry in industries:
    print(f"Processing industry: {industry}")
    codes = dict_data[industry]['종목코드'].unique()

    # 각 종목코드에 대해 모델을 학습하고 테스트
    for code in codes:
        print(f"Processing code: {code}")
        df = dict_data[industry][dict_data[industry]['종목코드'] == code].copy()

        df['날짜'] = pd.to_datetime(df['날짜'])
        df.set_index('날짜', inplace=True)

        selected_vars = df[['종가'] + sectors_features[industry]]  # 선택된 피처와 종가 사용

        # 윈도우 사이즈와 롤링 윈도우 적용
        moving_vars = selected_vars.rolling(window=window_size).mean().shift(1)

        # 예측 대상인 종가를 제외한 피처들만 사용
        moving_vars = moving_vars.drop('종가', axis=1)

        # 실제 종가를 추가
        moving_vars['종가'] = selected_vars['종가']

        moving_vars = moving_vars.dropna()

        train = moving_vars[(moving_vars.index >= '2021-06-01') & (moving_vars.index <= '2022-12-31')]
        test = moving_vars[(moving_vars.index >= '2023-05-25') & (moving_vars.index <= '2023-05-29')]

        # 테스트 데이터셋이 비어있지 않은 경우에만 모델 학습 및 예측을 수행
        if not test.empty:
            X_train = train.drop('종가', axis=1)
            y_train = train['종가']
            X_test = test.drop('종가', axis=1)
            y_test = test['종가']

            scaler = MinMaxScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            model = xgb.XGBRegressor(objective='reg:squarederror')

            print(f'Training model for code {code}...')
            model.fit(X_train_scaled, y_train)

            print(f'Predicting for code {code}...')
            predictions = model.predict(X_test_scaled)

            # 5월30일의 예측 종가와 5일간의 평균 종가 차이 계산
            average_close = y_test.mean()
            difference = average_close - predictions[-1]

            print(f"Code: {code}, Predicted Close on May 30: {predictions[-1]}, Average Close from May 25 to May 29: {average_close}, Difference: {difference}")

            # 결과를 데이터프레임에 추가
            result_row = pd.DataFrame({'종목코드': [code], 'Difference': [difference], '예측종가_5월30일': [predictions[-1]]})
            result_row['Rank'] = result_row['Difference'].rank(ascending=False)

            result_df = pd.concat([result_df, result_row])


# 결과를 데이터프레임을 CSV 파일로 저장
result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/result.csv', index=False) #=>여기서의 Rank는 무시 (1로 채워짐)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Predicting for code 002150...
Code: 002150, Predicted Close on May 30: 10544.9609375, Average Close from May 25 to May 29: 8725.0, Difference: -1819.9609375
Processing code: 102260
Training model for code 102260...
Predicting for code 102260...
Code: 102260, Predicted Close on May 30: 4675.04638671875, Average Close from May 25 to May 29: 4605.0, Difference: -70.04638671875
Processing code: 032350
Training model for code 032350...
Predicting for code 032350...
Code: 032350, Predicted Close on May 30: 12298.8916015625, Average Close from May 25 to May 29: 11260.0, Difference: -1038.8916015625
Processing code: 286940
Training model for code 286940...
Predicting for code 286940...
Code: 286940, Predicted Close on May 30: 27400.759765625, Average Close from May 25 to May 29: 27900.0, Difference: 499.240234375
Processing code: 007120
Training model for code 007120...
Predicting for code 007120...
Code: 007120, Predicted Close on May 30: 1923.95166015625, 

#추가

In [ ]:
# 주가 대비 변동률 계산
result_df['주가대비변동률'] = result_df['Difference'] / result_df['예측종가_5월30일']


# 순위 매기기
result_df['순위'] = result_df['주가대비변동률'].rank(ascending=False)

# 결과를 CSV 파일로 저장
result_df.to_csv('result_with_rank.csv', index=False)

In [ ]:
# result_df.drop(['Difference', '예측종가_5월30일', 'Rank', '주가대비변동률'], axis=1, inplace=True)
# result_df['종목코드'] = 'A' + result_df['종목코드']
# result_df['순위'] = result_df['순위'].astype(int)
# result_df

,종목코드,순위
0,A060720,1183
0,A091340,1415
0,A151910,1990
0,A190510,270
0,A267320,770
...,...,...
0,A298050,324
0,A298020,715
0,A298000,876
0,A093370,1283


In [ ]:
result_df = pd.read_csv('result_with_rank.csv')
result_df

,종목코드,Difference,예측종가_5월30일,Rank,주가대비변동률,순위
0,60720,-387.226562,20712.2270,1.0,-0.018696,1183.0
1,91340,-151.173340,4281.1733,1.0,-0.035311,1415.0
2,151910,-393.990112,909.4901,1.0,-0.433199,1990.0
3,190510,1045.368164,15009.6320,1.0,0.069646,270.0
4,267320,25.896729,3429.1033,1.0,0.007552,770.0
...,...,...,...,...,...,...
1995,298050,24700.687500,422299.3000,1.0,0.058491,324.0
1996,298020,4409.437500,382590.5600,1.0,0.011525,715.0
1997,298000,60.835938,100339.1640,1.0,0.000606,876.0
1998,93370,-339.788086,13359.7880,1.0,-0.025434,1283.0


#하위200개 골라내기 ( +상위200개 골라내서 합치고, 나머지것들은 랜덤 순위)

In [ ]:
#주가대비변동률이 음수이고 5일간 평균 종가와 예측 종가의 차이가 큰 마이너스 값을 하위200개에 배치하고자 함
# 주가대비변동률이 음수이고 5일간 평균 종가와 예측 종가의 차이가 큰 마이너스 값인 경우를 선택
filtered_df = result_df[(result_df['주가대비변동률'] < 0) & (result_df['Difference'] < 0)]

# 순위를 매기기 위해 'Difference' 컬럼을 기준으로 내림차순 정렬
filtered_df = filtered_df.sort_values('Difference', ascending=False)

# 순위 컬럼 추가 및 순위 매김
filtered_df['순위'] = filtered_df['Difference'].rank(ascending=False)

# 결과 확인
print(filtered_df[['종목코드', 'Difference', '순위']])

        종목코드    Difference      순위
506    32860     -0.000977     1.0
342   226440     -0.011597     2.0
1140   57880     -0.023682     3.0
1300  269620     -0.043091     4.0
1317  150840     -0.044922     5.0
...      ...           ...     ...
1461   17390 -45637.140625  1099.0
1973    3240 -47927.375000  1100.0
936     3100 -50362.195312  1101.0
1846   51900 -51565.750000  1102.0
1702   10130 -72322.625000  1103.0

[1103 rows x 3 columns]


In [ ]:

# down 200에 하위 200개 저장
down_200 = filtered_df.tail(200)

# down 200의 순위를 다시 매김
down_200['Rank'] = range(1801, 2001)

# 최종 순위를 오름차순으로 정렬
final_ranking = pd.concat([filtered_df.head(len(filtered_df) - 200), down_200])

# 최종 순위를 하위 200개만 저장
final_ranking = final_ranking.tail(200)

# 결과 확인
final_ranking

<ipython-input-7-7460fe3fb49e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  down_200['Rank'] = range(1801, 2001)


,종목코드,Difference,예측종가_5월30일,Rank,주가대비변동률,순위
1423,270660,-1403.246094,17223.246,1801.0,-0.081474,904.0
1991,48410,-1405.691406,24030.691,1802.0,-0.058496,905.0
564,293490,-1406.664062,40031.664,1803.0,-0.035139,906.0
1532,129890,-1412.347168,6059.847,1804.0,-0.233066,907.0
91,16250,-1415.105469,26765.105,1805.0,-0.052871,908.0
...,...,...,...,...,...,...
1461,17390,-45637.140625,130037.140,1996.0,-0.350955,1099.0
1973,3240,-47927.375000,709927.400,1997.0,-0.067510,1100.0
936,3100,-50362.195312,76137.195,1998.0,-0.661466,1101.0
1846,51900,-51565.750000,608565.750,1999.0,-0.084733,1102.0


In [ ]:
final_ranking['Rank'] = final_ranking['Rank'].astype(int)
final_ranking

,종목코드,Difference,예측종가_5월30일,Rank,주가대비변동률,순위
1423,270660,-1403.246094,17223.246,1801,-0.081474,904.0
1991,48410,-1405.691406,24030.691,1802,-0.058496,905.0
564,293490,-1406.664062,40031.664,1803,-0.035139,906.0
1532,129890,-1412.347168,6059.847,1804,-0.233066,907.0
91,16250,-1415.105469,26765.105,1805,-0.052871,908.0
...,...,...,...,...,...,...
1461,17390,-45637.140625,130037.140,1996,-0.350955,1099.0
1973,3240,-47927.375000,709927.400,1997,-0.067510,1100.0
936,3100,-50362.195312,76137.195,1998,-0.661466,1101.0
1846,51900,-51565.750000,608565.750,1999,-0.084733,1102.0


In [ ]:
# Difference, 예측종가_5월30일, Rank, 주가대비변동률, 순위 컬럼 제거
final_ranking = final_ranking.drop(['Difference', '예측종가_5월30일','주가대비변동률', '순위'], axis=1)

# 결과 확인
final_ranking

,종목코드,Rank
1423,270660,1801
1991,48410,1802
564,293490,1803
1532,129890,1804
91,16250,1805
...,...,...
1461,17390,1996
1973,3240,1997
936,3100,1998
1846,51900,1999


In [ ]:
final_ranking['종목코드'] = final_ranking['종목코드'].astype(str).str.zfill(6)
final_ranking['종목코드'] = np.where(final_ranking['종목코드'].str.len() == 6, 'A' + final_ranking['종목코드'], final_ranking['종목코드'])
final_ranking

,종목코드,Rank
1423,A270660,1801
1991,A048410,1802
564,A293490,1803
1532,A129890,1804
91,A016250,1805
...,...,...
1461,A017390,1996
1973,A003240,1997
936,A003100,1998
1846,A051900,1999


In [ ]:

final_ranking.rename(columns={'Rank': '순위'}, inplace=True)

#상위200개 출력

In [ ]:
result_df

,종목코드,Difference,예측종가_5월30일,Rank,주가대비변동률,순위
0,60720,-387.226562,20712.2270,1.0,-0.018696,1183.0
1,91340,-151.173340,4281.1733,1.0,-0.035311,1415.0
2,151910,-393.990112,909.4901,1.0,-0.433199,1990.0
3,190510,1045.368164,15009.6320,1.0,0.069646,270.0
4,267320,25.896729,3429.1033,1.0,0.007552,770.0
...,...,...,...,...,...,...
1995,298050,24700.687500,422299.3000,1.0,0.058491,324.0
1996,298020,4409.437500,382590.5600,1.0,0.011525,715.0
1997,298000,60.835938,100339.1640,1.0,0.000606,876.0
1998,93370,-339.788086,13359.7880,1.0,-0.025434,1283.0


In [ ]:
# 주가대비변동률에 대한 조건 포함해서 5일간 평균 종가와 예측 종가 차이가 양수인것들 기준으로 순위매겨서 상위200개만 추출
# threshold를 적게 할수록 주가의 안정성이 높은 종목들이 상위권에 포함될 가능성이 높아짐

threshold = 0.1
result_df['rank'] = result_df[(result_df['주가대비변동률'] < threshold) & (result_df['Difference'] > 0)].rank(ascending=False)['Difference']


In [ ]:
# NaN 값을 가진 rank를 하위로 배치하고, rank를 오름차순으로 정렬
result_df['rank'] = result_df['rank'].fillna(result_df['rank'].max() + 1)
result_df['rank'] = result_df['rank'].astype(int)
result_df = result_df.sort_values(by='rank')

# 결과 확인
result_df.head(50)


,종목코드,Difference,예측종가_5월30일,Rank,주가대비변동률,순위,rank
1246,3920,29632.843750,477867.160,1.0,0.062011,1,1
1995,298050,24700.687500,422299.300,1.0,0.058491,2,2
1698,5490,20591.750000,342158.250,1.0,0.060182,3,3
796,352820,13319.484375,255180.520,1.0,0.052196,4,4
1328,140860,11729.250000,142920.750,1.0,0.082068,5,5
387,3030,8882.687500,159717.310,1.0,0.055615,6,6
814,1460,8819.187500,414930.800,1.0,0.021255,7,7
1847,51910,7561.187500,701438.800,1.0,0.010780,8,8
1854,11790,6943.515625,88256.484,1.0,0.078674,9,9
1478,361610,5737.710938,83312.290,1.0,0.068870,10,10


In [ ]:

# 상위 200개 행만 유지하고 나머지 행은 제거
result_df = result_df.head(200)
result_df


,종목코드,Difference,예측종가_5월30일,Rank,주가대비변동률,순위,rank
1246,3920,29632.843750,477867.160,1.0,0.062011,1,1
1995,298050,24700.687500,422299.300,1.0,0.058491,2,2
1698,5490,20591.750000,342158.250,1.0,0.060182,3,3
796,352820,13319.484375,255180.520,1.0,0.052196,4,4
1328,140860,11729.250000,142920.750,1.0,0.082068,5,5
...,...,...,...,...,...,...,...
1139,51380,474.505859,6880.494,1.0,0.068964,196,196
1929,18250,471.830078,22253.170,1.0,0.021203,197,197
1909,18290,471.791016,5808.209,1.0,0.081228,198,198
1436,44340,470.975586,10859.024,1.0,0.043372,199,199


In [ ]:
# Difference, 예측종가_5월30일, Rank, 주가대비변동률, 순위 컬럼 제거
result_df = result_df.drop(['Difference', '예측종가_5월30일', 'Rank', '주가대비변동률', '순위'], axis=1)

# 결과 확인
result_df


,종목코드,rank
1246,3920,1
1995,298050,2
1698,5490,3
796,352820,4
1328,140860,5
...,...,...
1139,51380,196
1929,18250,197
1909,18290,198
1436,44340,199


In [ ]:
result_df['종목코드'] = result_df['종목코드'].astype(str).str.zfill(6)
result_df['종목코드'] = np.where(result_df['종목코드'].str.len() == 6, 'A' + result_df['종목코드'], result_df['종목코드'])
result_df

,종목코드,rank
1246,A003920,1
1995,A298050,2
1698,A005490,3
796,A352820,4
1328,A140860,5
...,...,...
1139,A051380,196
1929,A018250,197
1909,A018290,198
1436,A044340,199


In [ ]:

result_df.rename(columns={'rank': '순위'}, inplace=True)
result_df